In [1]:
import pandas as pd
import numpy as np

In [2]:
# load crowdsourcing dataset from tsv file
df = pd.read_csv('../data/crowdsource/crowd_data.tsv', sep='\t', encoding='utf-8')

In [3]:
df

,HITId,HITTypeId,Title,Reward,AssignmentId,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
0,1,7QT,Is this triple correct or incorrect?,$0.50,1,2133ICYWE97,Submitted,60,99%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
1,1,7QT,Is this triple correct or incorrect?,$0.50,2,2133U7HKDLO,Submitted,40,40%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,yes,yes
2,1,7QT,Is this triple correct or incorrect?,$0.50,3,928UJANWZ12,Submitted,50,98%,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
3,1,7QT,Is this triple correct or incorrect?,$0.50,4,1726JMZQW,Submitted,80,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
4,1,7QT,Is this triple correct or incorrect?,$0.50,5,2134U7HKDMM,Submitted,2,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,61,9QT,Is this triple correct or incorrect?,$0.50,301,AALKMII97,Submitted,240,98%,wd:Q1288004,wdt:P1412,wd:Q13330,2.0,INCORRECT,Object,Q1860
301,61,9QT,Is this triple correct or incorrect?,$0.50,302,HHCKW1111,Submitted,200,80%,wd:Q1288004,wdt:P1412,wd:Q13330,1.0,CORRECT,NaN,NaN
302,61,9QT,Is this triple correct or incorrect?,$0.50,303,GGUI83657S,Submitted,120,85%,wd:Q1288004,wdt:P1412,wd:Q13330,1.0,CORRECT,NaN,NaN
303,61,9QT,Is this triple correct or incorrect?,$0.50,304,WWHL098SA43,Submitted,10,69%,wd:Q1288004,wdt:P1412,wd:Q13330,2.0,INCORRECT,NaN,NaN


In [4]:
# clean answers
for idx, lbl in enumerate(df['FixValue'].values):
    if str(lbl)[:3] == 'wd:':
        print(lbl)
        print(str(lbl)[3:])
        df['FixValue'][idx] = str(lbl)[3:]

wd:Q72
Q72
wd:Q94074
Q94074
wd:Q95073
Q95073


/tmp/ipykernel_53763/3954016013.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FixValue'][idx] = str(lbl)[3:]


In [5]:
# find and filter out malicious workers 
malicious_workers = {}
# filter based on inputs
for idx, lbl in enumerate(df['AnswerLabel'].values):
    if lbl == 'INCORRECT':
        if str(df['FixValue'][idx]) in ['Yes', 'I don\'t understand', '100']:
            print(str(df['FixPosition'][idx]))
            print(str(df['FixValue'][idx]))
            # store the malicious worker id and the question id
            if df['WorkerId'][idx] not in malicious_workers:
                malicious_workers[df['WorkerId'][idx]] = {}
                malicious_workers[df['WorkerId'][idx]]['Approval'] = df['LifetimeApprovalRate'][idx]
                malicious_workers[df['WorkerId'][idx]]['Task'] = []
                malicious_workers[df['WorkerId'][idx]]['Time'] = 0
            malicious_workers[df['WorkerId'][idx]]['Task'] = np.append(malicious_workers[df['WorkerId'][idx]]['Task'], df['HITId'][idx])
            malicious_workers[df['WorkerId'][idx]]['Time'] += df['WorkTimeInSeconds'][idx]
    else:
        if str(df['FixPosition'][idx]) not in ['NaN', '', 'nan']:
            print(str(df['FixPosition'][idx]))
            print(str(df['FixValue'][idx])) 
            if df['WorkerId'][idx] not in malicious_workers:
                malicious_workers[df['WorkerId'][idx]] = {}
                malicious_workers[df['WorkerId'][idx]]['Approval'] = df['LifetimeApprovalRate'][idx]
                malicious_workers[df['WorkerId'][idx]]['Task'] = []
                malicious_workers[df['WorkerId'][idx]]['Time'] = 0
            malicious_workers[df['WorkerId'][idx]]['Task'] = np.append(malicious_workers[df['WorkerId'][idx]]['Task'], df['HITId'][idx])
            malicious_workers[df['WorkerId'][idx]]['Time'] += df['WorkTimeInSeconds'][idx]
# filter based on working time
for idx, time in enumerate(df['WorkTimeInSeconds'].values):
    if time < 20 and str(df['FixValue'][idx]) in ['Yes', 'I don\'t understand', '100', '', 'nan', 'NaN']:
        print(str(df['FixPosition'][idx]))
        print(str(df['FixValue'][idx]))
        if df['WorkerId'][idx] not in malicious_workers:
            malicious_workers[df['WorkerId'][idx]] = {}
            malicious_workers[df['WorkerId'][idx]]['Approval'] = df['LifetimeApprovalRate'][idx]
            malicious_workers[df['WorkerId'][idx]]['Task'] = []
            malicious_workers[df['WorkerId'][idx]]['Time'] = 0
        malicious_workers[df['WorkerId'][idx]]['Task'] = np.append(malicious_workers[df['WorkerId'][idx]]['Task'], df['HITId'][idx])
        malicious_workers[df['WorkerId'][idx]]['Time'] += df['WorkTimeInSeconds'][idx]
# get average work time for each worker
for worker in malicious_workers:
    malicious_workers[worker]['Time'] /= len(malicious_workers[worker]['Task'])


yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
I don't understand
I don't understand
I don't understand
I don't understand
100
100
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
100
100
100
100
100
100
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [6]:
malicious_workers

{'2133U7HKDLO': {'Approval': '40%',
  'Task': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15., 16., 17., 18., 19., 20., 21.]),
  'Time': 41.0},
 'WWHL098SA43': {'Approval': '69%',
  'Task': array([42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54.,
         55., 56., 57., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
         52., 53., 54., 55., 56., 57., 58., 59., 60., 61.]),
  'Time': 10.0},
 'QZAHIFT8263': {'Approval': '40%',
  'Task': array([44., 58., 60., 61., 42., 43., 44., 45., 46., 47., 48., 49., 50.,
         51., 52., 54., 55., 56., 57., 58., 59., 60., 61.]),
  'Time': 2.0869565217391304},
 '2134U7HKDMM': {'Approval': '70%',
  'Task': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15., 16., 17., 18., 19., 20., 21.]),
  'Time': 2.0},
 'LPQMUDT6729': {'Approval': '40%',
  'Task': array([22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 34.,
         35., 36., 37., 38., 39

In [7]:
# drop malicious workers from the dataset
for worker in malicious_workers:
    df = df[df.WorkerId != worker]

In [8]:
df

,HITId,HITTypeId,Title,Reward,AssignmentId,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
0,1,7QT,Is this triple correct or incorrect?,$0.50,1,2133ICYWE97,Submitted,60,99%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
2,1,7QT,Is this triple correct or incorrect?,$0.50,3,928UJANWZ12,Submitted,50,98%,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
3,1,7QT,Is this triple correct or incorrect?,$0.50,4,1726JMZQW,Submitted,80,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
6,2,7QT,Is this triple correct or incorrect?,$0.50,7,2133ICYWE97,Submitted,120,99%,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,NaN,NaN
7,2,7QT,Is this triple correct or incorrect?,$0.50,8,928UJANWZ12,Submitted,60,98%,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,60,9QT,Is this triple correct or incorrect?,$0.50,297,HHCKW1111,Submitted,200,80%,wd:Q21060270,wdt:P27,wd:Q916,1.0,CORRECT,NaN,NaN
297,60,9QT,Is this triple correct or incorrect?,$0.50,298,GGUI83657S,Submitted,120,85%,wd:Q21060270,wdt:P27,wd:Q916,1.0,CORRECT,NaN,NaN
300,61,9QT,Is this triple correct or incorrect?,$0.50,301,AALKMII97,Submitted,240,98%,wd:Q1288004,wdt:P1412,wd:Q13330,2.0,INCORRECT,Object,Q1860
301,61,9QT,Is this triple correct or incorrect?,$0.50,302,HHCKW1111,Submitted,200,80%,wd:Q1288004,wdt:P1412,wd:Q13330,1.0,CORRECT,NaN,NaN


In [9]:
# split the dataset into different batches based on HITTypeId
df1 = df[df.HITTypeId == '7QT']
df2 = df[df.HITTypeId == '8QT']
df3 = df[df.HITTypeId == '9QT']

In [10]:
# get the inter-rate agreement for each question (HITId)
# inter-rate agreement is calculated by Fleiss' Kappa
# https://en.wikipedia.org/wiki/Fleiss%27_kappa
def inter_rate_agreement(df):
    # get the number of workers who answered the question
    n = len(df['WorkerId'].unique())
    #n = 3
    # get the number of labels
    k = len(df['AnswerLabel'].unique())
    # get the number of questions
    N = len(df['HITId'].unique())
    # get the number of labels for each question
    #N = df.groupby('HITId').size().values
    print(n, k, N)
    # get the number of labels for each question and each label
    Nij = np.zeros((N, k))
    for i in range(N):
        for j in range(k):
            Nij[i, j] = len(df[(df['HITId'] == df['HITId'].unique()[i]) & (df['AnswerLabel'] == df['AnswerLabel'].unique()[j])])
    # calculate the inter-rate agreement
    pi = np.sum(Nij * (Nij - 1), axis=1) / (n * (n - 1))
    pj = np.sum(Nij, axis=0) / (N * n)
    print(pi)
    print(pj)
    pbar = np.sum(pi) / N
    pbarE = np.sum(np.square(pj), axis=0)
    kappa = (pbar - pbarE) / (1 - pbarE)
    print(f"kappa = ({pbar} - {pbarE}) / (1 - {pbarE})")
    return kappa

In [12]:
# apply inter-rate agreement to the dataset (before filtering for malicious workers)
crowd_kappa_7QT = inter_rate_agreement(df1).round(2)
crowd_kappa_8QT = inter_rate_agreement(df2).round(2)
crowd_kappa_9QT = inter_rate_agreement(df3).round(2)

3 2 21
[0.33333333 1.         0.33333333 1.         1.         0.33333333
 0.33333333 1.         1.         0.33333333 0.33333333 1.
 0.33333333 1.         1.         0.33333333 0.33333333 1.
 0.33333333 0.33333333 0.33333333]
[0.52380952 0.47619048]
kappa = (0.6190476190476192 - 0.5011337868480725) / (1 - 0.5011337868480725)
3 2 20
[0.33333333 1.         0.33333333 0.33333333 1.         0.33333333
 1.         1.         1.         0.33333333 0.33333333 0.33333333
 1.         0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
 0.33333333 0.33333333]
[0.58333333 0.41666667]
kappa = (0.5333333333333334 - 0.513888888888889) / (1 - 0.513888888888889)
3 2 20
[0.33333333 1.         1.         0.33333333 1.         1.
 0.33333333 0.33333333 1.         1.         1.         0.33333333
 0.33333333 0.33333333 0.33333333 0.33333333 1.         0.33333333
 0.33333333 0.33333333]
[0.48333333 0.51666667]
kappa = (0.6000000000000001 - 0.5005555555555556) / (1 - 0.5005555555555556)


In [1]:
print(f"crowd_kappa_7QT = {crowd_kappa_7QT}")
print(f"crowd_kappa_8QT = {crowd_kappa_8QT}")
print(f"crowd_kappa_9QT = {crowd_kappa_9QT}")

NameError: name 'crowd_kappa_7QT' is not defined

In [13]:
# apply inter-rate agreement to the dataset (after filtering for malicious workers)
crowd_kappa_7QT = inter_rate_agreement(df1).round(2)
crowd_kappa_8QT = inter_rate_agreement(df2).round(2)
crowd_kappa_9QT = inter_rate_agreement(df3).round(2)

3 2 21
[0.33333333 1.         0.33333333 1.         1.         0.33333333
 0.33333333 1.         1.         0.33333333 0.33333333 1.
 0.33333333 1.         1.         0.33333333 0.33333333 1.
 0.33333333 0.33333333 0.33333333]
[0.52380952 0.47619048]
kappa = (0.6190476190476192 - 0.5011337868480725) / (1 - 0.5011337868480725)
3 2 20
[0.33333333 1.         0.33333333 0.33333333 1.         0.33333333
 1.         1.         1.         0.33333333 0.33333333 0.33333333
 1.         0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
 0.33333333 0.33333333]
[0.58333333 0.41666667]
kappa = (0.5333333333333334 - 0.513888888888889) / (1 - 0.513888888888889)
3 2 20
[0.33333333 1.         1.         0.33333333 1.         1.
 0.33333333 0.33333333 1.         1.         1.         0.33333333
 0.33333333 0.33333333 0.33333333 0.33333333 1.         0.33333333
 0.33333333 0.33333333]
[0.48333333 0.51666667]
kappa = (0.6000000000000001 - 0.5005555555555556) / (1 - 0.5005555555555556)


In [14]:
print(f"crowd_kappa_7QT = {crowd_kappa_7QT}")
print(f"crowd_kappa_8QT = {crowd_kappa_8QT}")
print(f"crowd_kappa_9QT = {crowd_kappa_9QT}")

crowd_kappa_7QT = 0.24
crowd_kappa_8QT = 0.04
crowd_kappa_9QT = 0.2


In [18]:
# get the number of tasks completed per minute for each batch
def get_task_per_minute(df):
    # get the number of tasks completed per minute
    task_per_minute = len(df['HITId'].unique()) / (df['WorkTimeInSeconds'].sum() / 60 / 3)
    return task_per_minute

In [19]:
throughput_7QT = get_task_per_minute(df1)
throughput_8QT = get_task_per_minute(df2)
throughput_9QT = get_task_per_minute(df3)

In [20]:
print(f"throughput_7QT = {throughput_7QT}")
print(f"throughput_8QT = {throughput_8QT}")
print(f"throughput_9QT = {throughput_9QT}")

throughput_7QT = 0.40623320795271356
throughput_8QT = 0.31729243786356426
throughput_9QT = 0.32134249754530037


In [104]:
# group by HITId
grouped = df.groupby('HITId')
# save majority vote answers to csv file
crowd_answers = {'HITId': [], 'kappa': [], 'label': [], 'support': [], 'Subject': [], 'Predicate': [], 'Object': [], 'FixPosition': [], 'FixValue': []}
for name, group in grouped:
    # get the majority vote
    majority_vote = group['AnswerLabel'].value_counts().idxmax()
    # check if the majority vote is not agreed upon
    if group['HITTypeId'].values[0] == '7QT':
        crowd_kappa = crowd_kappa_7QT
    elif group['HITTypeId'].values[0] == '8QT':
        crowd_kappa = crowd_kappa_8QT
    elif group['HITTypeId'].values[0] == '9QT':
        crowd_kappa = crowd_kappa_9QT
    print(name)
    print(group['AnswerLabel'].value_counts())
    print('Majority vote: ' + majority_vote)
    if len(group['FixPosition'].value_counts()) == 1:
        crowd_answers['HITId'] = np.append(crowd_answers['HITId'], name)
        crowd_answers['kappa'] = np.append(crowd_answers['kappa'], crowd_kappa)
        # append number of workers that voted for the majority vote
        crowd_answers['label'] = np.append(crowd_answers['label'], majority_vote)
        crowd_answers['support'] = np.append(crowd_answers['support'], group['AnswerLabel'].value_counts().max())
        crowd_answers['Subject'] = np.append(crowd_answers['Subject'], group['Input1ID'].values[0])
        crowd_answers['Predicate'] = np.append(crowd_answers['Predicate'], group['Input2ID'].values[0])
        crowd_answers['Object'] = np.append(crowd_answers['Object'], group['Input3ID'].values[0])
        crowd_answers['FixPosition'] = np.append(crowd_answers['FixPosition'], group['FixPosition'].value_counts().idxmax())
        print('Fix position: ' + group['FixPosition'].value_counts().idxmax())
    else:
        crowd_answers['HITId'] = np.append(crowd_answers['HITId'], name)
        crowd_answers['kappa'] = np.append(crowd_answers['kappa'], crowd_kappa)
        # append number of workers that voted for the majority vote
        crowd_answers['label'] = np.append(crowd_answers['label'], majority_vote)
        crowd_answers['support'] = np.append(crowd_answers['support'], group['AnswerLabel'].value_counts().max())
        crowd_answers['Subject'] = np.append(crowd_answers['Subject'], group['Input1ID'].values[0])
        crowd_answers['Predicate'] = np.append(crowd_answers['Predicate'], group['Input2ID'].values[0])
        crowd_answers['Object'] = np.append(crowd_answers['Object'], group['Input3ID'].values[0])
        crowd_answers['FixPosition'] = np.append(crowd_answers['FixPosition'], 'NA')
    if len(group['FixValue'].value_counts()) == 1:
        crowd_answers['FixValue'] = np.append(crowd_answers['FixValue'], group['FixValue'].value_counts().idxmax())
        print('Fix value: ' + group['FixValue'].value_counts().idxmax())
    else:
        crowd_answers['FixValue'] = np.append(crowd_answers['FixValue'], 'NA')
    print('')

1
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

2
CORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

3
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT
Fix position: Object
Fix value: 2014-02-18

4
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object

5
CORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

6
INCORRECT    2
CORRECT      1
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object
Fix value: 2019-02-24

7
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT
Fix position: Object

8
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object
Fix value: 698491348

9
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT

10
INCORRECT    2
CORRECT      1
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT

11
CORRECT      2


In [105]:
crowd_answers = pd.DataFrame(crowd_answers)
crowd_answers

,HITId,kappa,label,support,Subject,Predicate,Object,FixPosition,FixValue
0,1.0,0.24,CORRECT,2.0,wd:Q11621,wdt:P2142,792910554,NA,NA
1,2.0,0.24,CORRECT,3.0,wd:Q603545,wdt:P2142,4300000,NA,NA
2,3.0,0.24,CORRECT,2.0,wd:Q16911843,wdt:P577,2014-01-18,Object,2014-02-18
3,4.0,0.24,INCORRECT,3.0,wd:Q132863,wdt:P2142,969023261,Object,NA
4,5.0,0.24,CORRECT,3.0,wd:Q1628022,wdt:P577,1951-01-01,NA,NA
...,...,...,...,...,...,...,...,...,...
56,57.0,0.20,CORRECT,2.0,wd:Q223596,wdt:P1431,wd:Q457180,Object,Q181900
57,58.0,0.20,INCORRECT,3.0,wd:Q943992,wdt:P161,wd:Q160432,NA,NA
58,59.0,0.20,CORRECT,2.0,wd:Q1893555,wdt:P272,wd:Q48784114,Object,Q7488442
59,60.0,0.20,CORRECT,2.0,wd:Q21060270,wdt:P27,wd:Q916,Object,Q884


In [106]:
# save the majority vote answers to csv file
crowd_answers.to_csv('../data/crowdsource/crowd_answers.csv', index=False)

# Triples to Correct in KG

In [97]:
# majority vote, show results that are not agreed upon
# group by HITId
grouped = df.groupby('HITId')
# iterate over groups
triples_to_correct = {'HITId': [], 'kappa': [], 'support': [], 'Subject': [], 'Predicate': [], 'Object': [], 'FixPosition': [], 'FixValue': []}
for name, group in grouped:
    # get the majority vote
    majority_vote = group['AnswerLabel'].value_counts().idxmax()
    # check if the majority vote is not agreed upon
    if group['HITTypeId'].values[0] == '7QT':
        crowd_kappa = crowd_kappa_7QT
    elif group['HITTypeId'].values[0] == '8QT':
        crowd_kappa = crowd_kappa_8QT
    elif group['HITTypeId'].values[0] == '9QT':
        crowd_kappa = crowd_kappa_9QT
    print(name)
    print(group['AnswerLabel'].value_counts())
    print('Majority vote: ' + majority_vote)
    if len(group['FixPosition'].value_counts()) == 1:
        if majority_vote == 'INCORRECT':
            triples_to_correct['HITId'] = np.append(triples_to_correct['HITId'], name)
            triples_to_correct['kappa'] = np.append(triples_to_correct['kappa'], crowd_kappa)
            # append number of workers that voted for the majority vote
            triples_to_correct['support'] = np.append(triples_to_correct['support'], group['AnswerLabel'].value_counts().max())
            triples_to_correct['Subject'] = np.append(triples_to_correct['Subject'], group['Input1ID'].values[0])
            triples_to_correct['Predicate'] = np.append(triples_to_correct['Predicate'], group['Input2ID'].values[0])
            triples_to_correct['Object'] = np.append(triples_to_correct['Object'], group['Input3ID'].values[0])
            triples_to_correct['FixPosition'] = np.append(triples_to_correct['FixPosition'], group['FixPosition'].value_counts().idxmax())
        print('Fix position: ' + group['FixPosition'].value_counts().idxmax())
    else:
        if majority_vote == 'INCORRECT':
            triples_to_correct['HITId'] = np.append(triples_to_correct['HITId'], name)
            triples_to_correct['kappa'] = np.append(triples_to_correct['kappa'], crowd_kappa)
            # append number of workers that voted for the majority vote
            triples_to_correct['support'] = np.append(triples_to_correct['support'], group['AnswerLabel'].value_counts().max())
            triples_to_correct['Subject'] = np.append(triples_to_correct['Subject'], group['Input1ID'].values[0])
            triples_to_correct['Predicate'] = np.append(triples_to_correct['Predicate'], group['Input2ID'].values[0])
            triples_to_correct['Object'] = np.append(triples_to_correct['Object'], group['Input3ID'].values[0])
            triples_to_correct['FixPosition'] = np.append(triples_to_correct['FixPosition'], 'NA')
    if len(group['FixValue'].value_counts()) == 1:
        if majority_vote == 'INCORRECT':
            triples_to_correct['FixValue'] = np.append(triples_to_correct['FixValue'], group['FixValue'].value_counts().idxmax())
        print('Fix value: ' + group['FixValue'].value_counts().idxmax())
    else:
        if majority_vote == 'INCORRECT':
            triples_to_correct['FixValue'] = np.append(triples_to_correct['FixValue'], 'NA')
    print('')

1
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

2
CORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

3
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT
Fix position: Object
Fix value: 2014-02-18

4
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object

5
CORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

6
INCORRECT    2
CORRECT      1
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object
Fix value: 2019-02-24

7
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT
Fix position: Object

8
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object
Fix value: 698491348

9
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT

10
INCORRECT    2
CORRECT      1
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT

11
CORRECT      2


In [98]:
triples_to_correct

{'HITId': array([ 4.,  6.,  8.,  9., 10., 14., 17., 19., 20., 21., 22., 23., 24.,
        26., 27., 29., 32., 34., 35., 36., 38., 40., 41., 42., 43., 44.,
        45., 46., 47., 58.]),
 'kappa': array([0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.04,
        0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
        0.04, 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 ]),
 'support': array([3., 2., 3., 3., 2., 3., 2., 2., 2., 2., 2., 3., 2., 3., 2., 3., 2.,
        3., 2., 2., 2., 2., 2., 2., 3., 3., 2., 3., 3., 3.]),
 'Subject': array(['wd:Q132863', 'wd:Q48313910', 'wd:Q217010', 'wd:Q4993462',
        'wd:Q7033842', 'wd:Q1004', 'wd:Q16101952', 'wd:Q15715406',
        'wd:Q15239622', 'wd:Q708135', 'wd:Q1720855', 'wd:Q598752',
        'wd:Q427386', 'wd:Q1410031', 'wd:Q20001199', 'wd:Q639070',
        'wd:Q2893887', 'wd:Q1039395', 'wd:Q202211', 'wd:Q28974159',
        'wd:Q885281', 'wd:Q6548158', 'wd:Q1123629', 'wd:Q1780602',
        'wd:Q610633', 'wd:Q2188914', 

In [99]:
triples_to_correct = pd.DataFrame(triples_to_correct)

In [100]:
triples_to_correct

,HITId,kappa,support,Subject,Predicate,Object,FixPosition,FixValue
0,4.0,0.24,3.0,wd:Q132863,wdt:P2142,969023261,Object,NA
1,6.0,0.24,2.0,wd:Q48313910,wdt:P577,2018-02-24,Object,2019-02-24
2,8.0,0.24,3.0,wd:Q217010,wdt:P2142,698491347,Object,698491348
3,9.0,0.24,3.0,wd:Q4993462,wdt:P577,2008-08-29,NA,NA
4,10.0,0.24,2.0,wd:Q7033842,wdt:P577,1996-06-01,NA,NA
5,14.0,0.24,3.0,wd:Q1004,ddis:indirectSubclassOf,wd:Q47461344,NA,NA
6,17.0,0.24,2.0,wd:Q16101952,ddis:indirectSubclassOf,wd:Q95074,Object,Q95073
7,19.0,0.24,2.0,wd:Q15715406,ddis:indirectSubclassOf,wd:Q27096213,NA,NA
8,20.0,0.24,2.0,wd:Q15239622,ddis:indirectSubclassOf,wd:Q27096213,NA,NA
9,21.0,0.24,2.0,wd:Q708135,ddis:indirectSubclassOf,wd:Q618779,NA,NA


In [101]:
# save triples to correct to csv
triples_to_correct.to_csv('../data/crowdsource/triples_to_correct.csv', index=False)

# Correct Knowledge Graph

In [2]:
import pandas as pd
# import triples to correct
#triples_to_correct = pd.read_csv('../data/crowdsource/triples_to_correct.csv')
triples_to_correct = pd.read_csv('../data/crowdsource/crowd_answers.csv')

## Crowd answer inspection

In [17]:
# load the embeddings
import os
import numpy as np
os.chdir('../data')
os.chdir('./ddis-graph-embeddings')
entity_emb = np.load('./entity_embeds.npy')
relation_emb = np.load('./relation_embeds.npy')
entity_file = './entity_ids.del'
relation_file = './relation_ids.del'

In [18]:
# load the dictionaries
import csv
with open(entity_file, 'r') as ifile:
    ent2id = {rdflib.term.URIRef(ent): int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
    id2ent = {v: k for k, v in ent2id.items()}
with open(relation_file, 'r') as ifile:
    rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}
    id2rel = {v: k for k, v in rel2id.items()}

In [19]:
ent2lbl = {ent: str(lbl) for ent, lbl in fixed_graph.subject_objects(RDFS.label)}
lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}
rel2lbl = {rel: str(lbl) for rel, lbl in fixed_graph.subject_objects(RDFS.label)}
lbl2rel = {lbl: rel for rel, lbl in rel2lbl.items()}

In [21]:
triples_to_correct['Subject']

0        Q11621
1       Q603545
2     Q16911843
3       Q132863
4      Q1628022
        ...    
56      Q223596
57      Q943992
58     Q1893555
59    Q21060270
60     Q1288004
Name: Subject, Length: 61, dtype: object

In [23]:
# transform RDF entries to string
for i in range(len(triples_to_correct)):
    try:
        triples_to_correct['Subject'].iloc[i] = ent2lbl[WD[triples_to_correct['Subject'].iloc[i]]]
        triples_to_correct['Predicate'].iloc[i] = rel2lbl[WDT[triples_to_correct['Predicate'].iloc[i]]]
        triples_to_correct['Object'].iloc[i] = ent2lbl[WD[triples_to_correct['Object'].iloc[i]]]
    except:
        print(triples_to_correct['Subject'].iloc[i], triples_to_correct['Predicate'].iloc[i], triples_to_correct['Object'].iloc[i])

http://www.wikidata.org/entity/E.T. the Extra-Terrestrial does not look like a valid URI, trying to serialize this will break.
/tmp/ipykernel_72050/885766428.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  triples_to_correct['Subject'].iloc[i] = ent2lbl[WD[triples_to_correct['Subject'].iloc[i]]]
/tmp/ipykernel_72050/885766428.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  triples_to_correct['Predicate'].iloc[i] = rel2lbl[WDT[triples_to_correct['Predicate'].iloc[i]]]
/tmp/ipykernel_72050/885766428.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

E.T. the Extra-Terrestrial box office 792910554
Peaceful Warrior box office 4300000
God Help the Girl publication date 2014-01-18
Finding Nemo box office 969023261
The Browning Version publication date 1951-01-01
The Party's Just Beginning publication date 2018-02-24
The Princess and the Frog box office 267000000
The Twilight Saga: Eclipse box office 698491347
The Candidate publication date 2008-08-29
Nightjohn publication date 1996-06-01
station building indirectSubclassOf Q27096213
film organization indirectSubclassOf Q43229
fictional princess indirectSubclassOf Q95074
comics indirectSubclassOf Q47461344
children's book indirectSubclassOf Q47461344
literary pentalogy indirectSubclassOf Q7725310
supervillain team indirectSubclassOf Q95074
film organization indirectSubclassOf Q43229
neighbourhood of Helsinki indirectSubclassOf Q27096213
disputed territory indirectSubclassOf Q27096213
Silver Bear indirectSubclassOf Q618779
Kaboom publication date 2010-10-01
Like Crazy publication date 2

## Triples to Correct and KG manipulation

In [20]:
triples_to_correct

,HITId,kappa,label,support,Subject,Predicate,Object,FixPosition,FixValue
0,1,0.24,CORRECT,2,Q11621,P2142,792910554,NaN,NaN
1,2,0.24,CORRECT,3,Q603545,P2142,4300000,NaN,NaN
2,3,0.24,CORRECT,2,Q16911843,P577,2014-01-18,Object,2014-02-18
3,4,0.24,INCORRECT,3,Q132863,P2142,969023261,Object,NaN
4,5,0.24,CORRECT,3,Q1628022,P577,1951-01-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...
56,57,0.20,CORRECT,2,Q223596,P1431,Q457180,Object,Q181900
57,58,0.20,INCORRECT,3,Q943992,P161,Q160432,Object,Q7360827
58,59,0.20,CORRECT,2,Q1893555,P272,Q48784114,Object,Q7488442
59,60,0.20,CORRECT,2,Q21060270,P27,Q916,Object,Q884


In [ ]:
# save triples to correct to csv
triples_to_correct.to_csv('../data/crowdsource/crowd_answers_labels.csv', index=False)

In [42]:
# import fixed knowledge graph from pickle
import pickle
with open('../data/fixed_graph.pickle', 'rb') as f:
    fixed_graph = pickle.load(f)

In [9]:
import rdflib
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, OWL, XSD

# define some prefixes
WD = rdflib.Namespace('http://www.wikidata.org/entity/')
WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
DDIS = rdflib.Namespace('http://ddis.ch/atai/')
RDFS = rdflib.namespace.RDFS
SCHEMA = rdflib.Namespace('http://schema.org/')

In [47]:
len(fixed_graph)

2056795

In [38]:
triples = fixed_graph.triples((None, None, None))

In [44]:
# delete incorrect triples from the fixed knowledge graph
for index, row in triples_to_correct.iterrows():
    try:
        if row['Object'][:1] == 'Q':
            obj = WD[(row['Object'])]
        else:
            obj = Literal(row['Object'])
        if row['Predicate'] == 'indirectSubclassOf':
            print(WD[row['Subject']], DDIS.indirectSubclassOf, obj)
            fixed_graph.remove((WD[row['Subject']], DDIS.indirectSubclassOf, obj))
        else:
            print(WD[row['Subject']], WDT[row['Predicate']], obj)
            fixed_graph.remove((WD[row['Subject']], WDT[row['Predicate']], obj))
    except:
        print(f"Error: {row}")

http://www.wikidata.org/entity/Q11621 http://www.wikidata.org/prop/direct/P2142 792910554
http://www.wikidata.org/entity/Q603545 http://www.wikidata.org/prop/direct/P2142 4300000
http://www.wikidata.org/entity/Q16911843 http://www.wikidata.org/prop/direct/P577 2014-01-18
http://www.wikidata.org/entity/Q132863 http://www.wikidata.org/prop/direct/P2142 969023261
http://www.wikidata.org/entity/Q1628022 http://www.wikidata.org/prop/direct/P577 1951-01-01
http://www.wikidata.org/entity/Q48313910 http://www.wikidata.org/prop/direct/P577 2018-02-24
http://www.wikidata.org/entity/Q171300 http://www.wikidata.org/prop/direct/P2142 267000000
http://www.wikidata.org/entity/Q217010 http://www.wikidata.org/prop/direct/P2142 698491347
http://www.wikidata.org/entity/Q4993462 http://www.wikidata.org/prop/direct/P577 2008-08-29
http://www.wikidata.org/entity/Q7033842 http://www.wikidata.org/prop/direct/P577 1996-06-01
http://www.wikidata.org/entity/Q1339195 http://ddis.ch/atai/indirectSubclassOf http://

In [83]:
for index, row in triples_to_correct.iterrows():
    if str(row['Object'])[:1] == 'Q':
        obj = WD[(row['Object'])]
    else:
        obj = Literal(row['Object'])
        print(obj)
    if str(row['FixValue'])[:1] == 'Q':
        obj_fix_q = WD[(row['FixValue'])]
    else:
        obj_fix_q = Literal(row['FixValue'])
        print(obj_fix_q)
    if str(row['FixValue'])[:1] == 'P':
        obj_fix_p = WDT[(row['FixValue'])]
    else:
        obj_fix_p = Literal(row['FixValue'])
        print(obj_fix_p)

792910554
nan
nan
4300000
nan
nan
2014-01-18
2014-02-18
2014-02-18
969023261
nan
nan
1951-01-01
nan
nan
2018-02-24
2019-02-24
2019-02-24
267000000
nan
nan
698491347
698491348
698491348
2008-08-29
nan
nan
1996-06-01
nan
nan
Q72
nan
nan
Q94074
nan
nan
nan
nan
nan
nan
Q95073
nan
nan
nan
nan
nan
nan
nan
nan
2010-10-01
2010-01-01
2010-01-01
2001-01-01
2011-01-01
2011-01-01
176997186
176997168
176997168
1982-01-01
1992-01-01
1992-01-01
2010-10-01
2011-01-01
2011-01-01
863756050
863756051
863756051
511200000
nan
nan
Q5423258
2015-01-05
nan
nan
10696220
10696210
10696210
Q1168152
2015-06-27
2015-08-27
2015-08-27
Q908556
Q1471671
Q1722254
Q15052538
Q409022
Q27703272
P106
P161
P344
P58
Q17350908
Q3194791
Q28732985
P58
Q268905
Q16795448
nan
nan
nan
nan
nan
nan
P27
P19
P17
P19
Q181900
Q7360827
Q7488442
Q884
Q1860


In [46]:
# add correct triples to the fixed knowledge graph
for index, row in triples_to_correct.iterrows():
    try:
        if str(row['Object'])[:1] == 'Q':
            obj = WD[(row['Object'])]
        else:
            obj = Literal(row['Object'])
            print(obj)
        if str(row['FixValue'])[:1] == 'Q':
            obj_fix_q = WD[(row['FixValue'])]
        else:
            obj_fix_q = Literal(row['FixValue'])
            print(obj_fix_q)
        if str(row['FixValue'])[:1] == 'P':
            obj_fix_p = WDT[(row['FixValue'])]
        else:
            obj_fix_p = Literal(row['FixValue'])
            print(obj_fix_p)
        if row['label'] == 'INCORRECT':
            if str(row['FixValue']) not in ['NA', 'NaN', 'nan']:
                if row['Predicate'] == 'indirectSubclassOf':
                    if row['FixPosition'] == 'Subject':
                        fixed_graph.add((obj_fix_q, DDIS.indirectSubclassOf, WD[(row['Object'])]))
                    elif row['FixPosition'] == 'Object':
                        fixed_graph.add((WD[row['Subject']], DDIS.indirectSubclassOf, obj_fix_q))
                    elif row['FixPosition'] == 'Predicate':
                        fixed_graph.add((WD[row['Subject']], obj_fix_p, WD[(row['Object'])]))
                    else:
                        print("unknown fix position")
                else:
                    if row['FixPosition'] == 'Subject':
                        fixed_graph.add((obj_fix_q, WDT[row['Predicate']], WD[(row['Object'])]))
                    elif row['FixPosition'] == 'Object':
                        fixed_graph.add((WD[row['Subject']], WDT[row['Predicate']], obj_fix_q))
                    elif row['FixPosition'] == 'Predicate':
                        fixed_graph.add((WD[row['Subject']], obj_fix_p, WD[(row['Object'])]))
                    else:
                        print(f"Unknown fix position: {row}")
            else:
                print(f"no fix value: {row}")
        else:
            if row['Predicate'] == 'indirectSubclassOf':
                fixed_graph.add((WD[row['Subject']], DDIS[row['Predicate']], obj))
            else:
                fixed_graph.add((WD[row['Subject']], WDT[row['Predicate']], obj))
    except:
        print(f"Error: {row}")

792910554
nan
nan
4300000
nan
nan
2014-01-18
2014-02-18
2014-02-18
969023261
nan
nan
no fix value: HITId                  4
kappa               0.24
label          INCORRECT
support                3
Subject          Q132863
Predicate          P2142
Object         969023261
FixPosition       Object
FixValue             NaN
Name: 3, dtype: object
1951-01-01
nan
nan
2018-02-24
2019-02-24
2019-02-24
267000000
nan
nan
698491347
698491348
698491348
2008-08-29
nan
nan
no fix value: HITId                   9
kappa                0.24
label           INCORRECT
support                 3
Subject          Q4993462
Predicate            P577
Object         2008-08-29
FixPosition           NaN
FixValue              NaN
Name: 8, dtype: object
1996-06-01
nan
nan
no fix value: HITId                  10
kappa                0.24
label           INCORRECT
support                 2
Subject          Q7033842
Predicate            P577
Object         1996-06-01
FixPosition           NaN
FixValue              

In [118]:
# save fixed knowledge graph to pickle
with open('../data/fixed_graph_crowd.pickle', 'wb') as f:
    pickle.dump(fixed_graph, f)